In [206]:
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import base64
from io import BytesIO

outputs = [
    "1000000000",  # 1
    "0100000000",  # 2
    "0010000000",  # 3
    "0001000000",  # 4
    "0000100000",  # 5
    "0000010000",  # 6
    "0000001000",  # 7
    "0000000100",  # 8
    "0000000010",  # 9
    "0000000001",  # 0
]

In [207]:
class NeuralNetwork(nn.Module):
    def __init__(self, nnets: int):
        super(NeuralNetwork, self).__init__()
        # first layer nnets neurons and tanh activation
        self.fc1 = nn.Linear(20, nnets)
        self.fc1.activation = nn.Tanh()
        # second layer 10 neurons and linear activation
        self.fc2 = nn.Linear(nnets, 10)
        self.model = nn.Sequential(self.fc1, self.fc2)

    def forward(self, x):
        x = self.model(x)
        return x

In [208]:
def read_tiles_from_image(image_path: str) -> List[List[int]]:
    # open the image "numeros.png"
    img = Image.open(image_path)
    # the image is a 40 x 5 image
    # split the image into 10 4 x 5 images

    # convert the image to black and white
    img = img.convert("1")

    numbers = []
    for i in range(10):
        # crop the image
        numbers.append(img.crop((i * 4, 0, (i + 1) * 4, 5)))

    # create a list of bit strips of the numbers
    # the index of the list is the number
    numbers_list = []
    for i in range(10):
        numbers_list.append([])
        for j in range(5):
            for k in range(4):
                numbers_list[i].append(0 if numbers[i].getpixel((k, j)) == 255 else 1)

    print("Pixels of the numbers in the image:")
    for i in numbers_list:
        print("".join(str(x) for x in i))

    return numbers_list

In [209]:
def bits_to_img(bits: List[int]) -> Image:
    img = Image.new("1", (4, 5))
    for i in range(5):
        for j in range(4):
            img.putpixel(
                (
                    j,
                    i,
                ),
                255 if bits[i * 4 + j] == 0 else 0,
            )
    return img


def img_to_base64(img: Image) -> str:
    with BytesIO() as buffer:
        img.save(buffer, "PNG")
        return base64.b64encode(buffer.getvalue()).decode()


def image_formatter(base64img: str) -> str:
    return f'<img style="display:inline-block; width: 50px; height: 50px; margin: 0px; image-rendering: pixelated;" src="data:image/png;base64,{base64img}">'


def bits_tensor_to_printable(bits: List[int]) -> str:
    img = bits_to_img(bits)
    base64img = img_to_base64(img)
    return image_formatter(base64img)

In [210]:
def train_network(
    input_tensors: torch.Tensor,
    output_tensors: torch.Tensor,
    nnets: int,
    epochs: int,
    goal: float,
    learning_rate: float,
    momentum: float,
) -> Tuple[NeuralNetwork, pd.DataFrame]:
    # Mean Squared Loss
    criterion = nn.MSELoss()
    model = NeuralNetwork(nnets)
    # stochastic gradient descent
    optimizer = optim.SGD(  # Stochastic Gradient Descent/Gradient Descent with Momentum (TRAINGDM Equivalent)
        model.parameters(), lr=learning_rate, momentum=momentum
    )

    history_data = pd.DataFrame(
        columns=["nnets", "epoch", "goal", "learning_rate", "momentum"]
    )

    history_data = pd.concat(
        [
            history_data,
            pd.DataFrame(
                [{
                    "nnets": nnets,
                    "epoch": 0,
                    "goal": goal,
                    "learning_rate": learning_rate,
                    "momentum": momentum,
                }]
            ),
        ],
        ignore_index=True,
    )

    # Train the neural network
    for epoch in range(epochs):
        # Forward pass
        outputs = model(input_tensors)
        loss = criterion(outputs, output_tensors)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        history_data = pd.concat(
            [
                history_data,
                pd.DataFrame(
                    [{
                        "nnets": nnets,
                        "epoch": epoch,
                        "goal": goal,
                        "learning_rate": learning_rate,
                        "momentum": momentum,
                    }]
                ),
            ],
            ignore_index=True,
        )

        if epoch % 1000 == 0:
            print(
                f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.6f}, Learning Rate: {learning_rate}, Momentum: {momentum}"
            )

        if loss.item() < goal:
            break

    print(f"Final loss: {loss.item():.6f}")
    return model, history_data


In [211]:
def make_flip_n_bits(bits: torch.Tensor, n: int = 1) -> torch.Tensor:
    import random

    randomIdxs = random.sample(range(0, len(bits)), n)
    for i in randomIdxs:
        bits[i] = 1 if bits[i] == 0 else 0
    return bits

In [212]:
def test_network(
    model: NeuralNetwork,
    input_tensors: List[torch.Tensor],
    output_tensors: List[torch.Tensor],
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    # make a copy of the original input tensors
    original_input_tensors = input_tensors.clone()

    # create a dataframe to store the test results
    test_results_0 = pd.DataFrame(
        columns=["input", "output", "prediction", "correct", "n_wrong_bits"]
    )
    test_results_1 = test_results_0.copy()
    test_results_2 = test_results_0.copy()
    test_results_3 = test_results_0.copy()

    # TODO: find a better name to this function
    def this_test(
        model: NeuralNetwork,
        input_tensor: torch.Tensor,
        output_tensor: torch.Tensor,
        original_input_tensor,
        test_results,
        n_wrong_bits,
    ):
        with torch.no_grad():
            outputs = model(input_tensor)
            correctness = torch.eq(outputs, output_tensor)
            test_results = pd.concat(
                [
                    test_results,
                    pd.DataFrame(
                        [{
                            "input":original_input_tensor,
                            "output":output_tensor,
                            "prediction":outputs,
                            "correct":correctness,
                            "n_wrong_bits":n_wrong_bits,
                        }]
                    ),
                ],
                ignore_index=True,
            )

        return test_results

    # Test the neural network
    print("Testing the neural network")
    for i in range(len(input_tensors)):
        test_results_0 = this_test(
            model,
            input_tensors[i],
            output_tensors[i],
            original_input_tensors[i],
            test_results_0,
            0
        )

    input_tensors = original_input_tensors.clone()

    # flip 1 bit in each input and test again
    print("Testing the neural network with 1 bit flipped in each input")
    for i in range(len(input_tensors)):
        input_tensors[i] = make_flip_n_bits(input_tensors[i], 1)
    for i in range(len(input_tensors)):
        test_results_1 = this_test(
            model,
            input_tensors[i],
            output_tensors[i],
            original_input_tensors[i],
            test_results_0,
            1
        )

    input_tensors = original_input_tensors.clone()

    # flip 2 bits in each input and test again
    print("Testing the neural network with 2 bits flipped in each input")
    for i in range(len(input_tensors)):
        input_tensors[i] = make_flip_n_bits(input_tensors[i], 2)
    for i in range(len(input_tensors)):
        test_results_2 = this_test(
            model,
            input_tensors[i],
            output_tensors[i],
            original_input_tensors[i],
            test_results_0,
            2
        )

    input_tensors = original_input_tensors.clone()

    # flip 3 bits in each input and test again
    print("Testing the neural network with 3 bits flipped in each input")
    for i in range(len(input_tensors)):
        input_tensors[i] = make_flip_n_bits(input_tensors[i], 3)
    for i in range(len(input_tensors)):
        test_results_3 = this_test(
            model,
            input_tensors[i],
            output_tensors[i],
            original_input_tensors[i],
            test_results_0,
            3
        )

    return test_results_0, test_results_1, test_results_2, test_results_3


In [213]:
def tests(
    input_tensors: torch.Tensor, output_tensors: torch.Tensor
) -> Tuple[List[pd.DataFrame], List[pd.DataFrame]]:
    train_params = [
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.1, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.4, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.9, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.1, "momentum": 0.4},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.9, "momentum": 0.4},
    ]
    neural_networks = [15, 25, 35]

    train_history_data_frames: List[pd.DataFrame] = []

    test_results_data_frames: List[pd.DataFrame] = []

    for nnets in neural_networks:
        for params in train_params:
            epochs = params["epochs"]
            goal = params["goal"]
            learning_rate = params["learning_rate"]
            momentum = params["momentum"]

            # Train the neural network
            trained_model, model_history = train_network(
                input_tensors,
                output_tensors,
                nnets,
                epochs,
                goal,
                learning_rate,
                momentum,
            )
            train_history_data_frames.append(model_history)

            # Test the neural network
            (
                test_results_0,
                test_results_1,
                test_results_2,
                test_results_3,
            ) = test_network(trained_model, input_tensors, output_tensors)
            test_results_data_frames.append(test_results_0)
            test_results_data_frames.append(test_results_1)
            test_results_data_frames.append(test_results_2)
            test_results_data_frames.append(test_results_3)

    return train_history_data_frames, test_results_data_frames

In [214]:
data = read_tiles_from_image("numeros.png")

Pixels of the numbers in the image:
01001100010001001110
01101001001001001111
11100001001000011110
10101010111100100010
11111000111000011110
01111000111010010110
11110001001001000100
01101001011010010110
01101001011100011111
01101001100110010110


In [215]:
input_tensor = torch.tensor(data, dtype=torch.float)
input_tensor

tensor([[0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
         1., 0.],
        [0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
         1., 1.],
        [1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
         1., 0.],
        [1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
         1., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
         1., 0.],
        [0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
         1., 0.],
        [1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
         0., 0.],
        [0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
         1., 0.],
        [0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
         1., 1.],
        [0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
         1., 0.]])

In [216]:
output_tensor = torch.tensor(
    [list(map(int, outp)) for outp in outputs], dtype=torch.float
)
output_tensor

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [217]:
train_history_data_frames, test_results_data_frames = tests(
    input_tensor, output_tensor
)

Epoch [0/10000], Loss: 0.194956, Learning Rate: 0.1, Momentum: 0.0
Epoch [1000/10000], Loss: 0.007771, Learning Rate: 0.1, Momentum: 0.0
Epoch [2000/10000], Loss: 0.002919, Learning Rate: 0.1, Momentum: 0.0
Final loss: 0.000499
Testing the neural network
Testing the neural network with 1 bit flipped in each input
Testing the neural network with 2 bits flipped in each input
Testing the neural network with 3 bits flipped in each input
Epoch [0/10000], Loss: 0.242036, Learning Rate: 0.4, Momentum: 0.0
Final loss: 0.000495
Testing the neural network
Testing the neural network with 1 bit flipped in each input
Testing the neural network with 2 bits flipped in each input
Testing the neural network with 3 bits flipped in each input
Epoch [0/10000], Loss: 0.131779, Learning Rate: 0.9, Momentum: 0.0
Final loss: 0.000498
Testing the neural network
Testing the neural network with 1 bit flipped in each input
Testing the neural network with 2 bits flipped in each input
Testing the neural network wit

In [218]:
train_history_data_frames[0]

,nnets,epoch,goal,learning_rate,momentum
0,15,0,0.0005,0.1,0.0
1,15,0,0.0005,0.1,0.0
2,15,1,0.0005,0.1,0.0
3,15,2,0.0005,0.1,0.0
4,15,3,0.0005,0.1,0.0
...,...,...,...,...,...
2896,15,2895,0.0005,0.1,0.0
2897,15,2896,0.0005,0.1,0.0
2898,15,2897,0.0005,0.1,0.0
2899,15,2898,0.0005,0.1,0.0


In [219]:
test_results_data_frames[0]

,input,output,prediction,correct,n_wrong_bits
0,"[tensor(0.), tensor(1.), tensor(0.), tensor(0....","[tensor(1.), tensor(0.), tensor(0.), tensor(0....","[tensor(0.9972), tensor(0.0012), tensor(-0.005...","[tensor(False), tensor(False), tensor(False), ...",0
1,"[tensor(0.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(1.), tensor(0.), tensor(0....","[tensor(0.0017), tensor(0.9992), tensor(0.0039...","[tensor(False), tensor(False), tensor(False), ...",0
2,"[tensor(1.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(1.), tensor(0....","[tensor(-0.0101), tensor(0.0034), tensor(0.981...","[tensor(False), tensor(False), tensor(False), ...",0
3,"[tensor(1.), tensor(0.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(0.), tensor(1....","[tensor(-0.0017), tensor(0.0007), tensor(-0.00...","[tensor(False), tensor(False), tensor(False), ...",0
4,"[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0.0145), tensor(-0.0051), tensor(0.027...","[tensor(False), tensor(False), tensor(False), ...",0
5,"[tensor(0.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(-0.0158), tensor(0.0051), tensor(-0.03...","[tensor(False), tensor(False), tensor(False), ...",0
6,"[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0.0004), tensor(0.0002), tensor(0.0009...","[tensor(False), tensor(False), tensor(False), ...",0
7,"[tensor(0.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0.0120), tensor(-0.0042), tensor(0.021...","[tensor(False), tensor(False), tensor(False), ...",0
8,"[tensor(0.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(-0.0037), tensor(0.0016), tensor(-0.00...","[tensor(False), tensor(False), tensor(False), ...",0
9,"[tensor(0.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0.0031), tensor(-0.0010), tensor(0.005...","[tensor(False), tensor(False), tensor(False), ...",0


In [220]:
def tensor_to_string(tensor: torch.Tensor) -> str:
    return "".join(map(str, map(int, tensor)))

In [221]:
from IPython.display import HTML

result = test_results_data_frames[0].copy()
# create a new image column with the image tag
result['image'] = result['input'].apply(lambda x: bits_tensor_to_printable(x))
result['input'] = result['input'].apply(lambda x: tensor_to_string(x))
result['prediction'] = result['prediction'].apply(lambda x: tensor_to_string(x))
result['output'] = result['output'].apply(lambda x: tensor_to_string(x))
result['correct'] = result['correct'].apply(lambda x: "Yes" if all(x) else "No")
html_code = result.to_html(escape=False)

# save the html code to a file
#with open("test_results.html", "w") as f:
#    f.write(html_code)

HTML(html_code)

,input,output,prediction,correct,n_wrong_bits,image
0,01001100010001001110,1000000000,0000000000,No,0,
1,01101001001001001111,0100000000,0000000000,No,0,
2,11100001001000011110,0010000000,0000000000,No,0,
3,10101010111100100010,0001000000,0001000000,No,0,
4,11111000111000011110,0000100000,0000000000,No,0,
5,01111000111010010110,0000010000,0000000000,No,0,
6,11110001001001000100,0000001000,0000001000,No,0,
7,01101001011010010110,0000000100,0000000000,No,0,
8,01101001011100011111,0000000010,0000000000,No,0,
9,01101001100110010110,0000000001,0000000000,No,0,
